In [ ]:
# loading the json files
import json
import pandas as pd
from pathlib import Path

data_path = Path("spotify-analytics\data\raw\spotify_export")

records = []
files = sorted(data_path.glob("Streaming_History_Audio*.json"))
print(f"Found {len(files)} files in {data_path}")
for file in files:
    print("Reading", file.name)
    with open(file, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)
        except Exception as e:
            print("Failed to parse", file.name, ":", e)
            continue
        # If the JSON is an object, try to find the list inside it
        if isinstance(data, dict):
            for key in ("StreamingHistory", "streaming_history", "items", "sessions"):
                if key in data and isinstance(data[key], list):
                    data = data[key]
                    break
            else:
                lists = [v for v in data.values() if isinstance(v, list)]
                if lists:
                    data = lists[0]
                else:
                    print("No list found in", file.name, "- skipping")
                    continue
        if not isinstance(data, list):
            print("Unexpected JSON structure in", file.name, "- skipping")
            continue
        records.extend(data)

df = pd.DataFrame(records)
print("Loaded records:", len(records))
df.head()

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3508491489.py, line 6)

In [29]:
df.columns

RangeIndex(start=0, stop=0, step=1)